# 0. Get/Install Dependencies

In [1]:
%%capture
!wget --no-check-certificate http://daas.res.ibm.com/~mudhakar/time_series/master/tspy-2.0.5.0.tar.gz -O ./tspy-2.0.5.0.tar.gz
!pip install -I tspy-2.0.5.0.tar.gz
!pip install pandas numpy py4j

In [2]:
import tspy

# 1. I/O

tspy provides support for ingestion from in memory lists, dicts, pandas dataframes as well as from other formats such as kafka and mq.

if a reader that you need does not exist, it can be created through the use of implementing a TimeSeriesReader

## 1.1 In Memory List

In [3]:
ts = tspy.time_series.list([1.0,2.0,3.0])
print(ts)

TimeStamp: 0     Value: 1.0
TimeStamp: 1     Value: 2.0
TimeStamp: 2     Value: 3.0


## 1.2 Pandas DataFrame

### 1.2.0 data preparation

In [4]:
import numpy as np
import pandas as pd

data = np.array([['', 'key', 'timestamp', "value"],
                 ['', "a", 1, 27],
                 ['', "b", 3, 4],
                 ['', "a", 5, 17],
                 ['', "a", 3, 7],
                 ['', "b", 2, 45]
                ])

df = pd.DataFrame(data=data[1:, 1:],
                  index=data[1:, 0],
                  columns=data[0, 1:]).astype(dtype={'key': 'object', 'timestamp': 'int64', 'value': 'float64'})
df.head()

key  timestamp  value
   a          1   27.0
   b          3    4.0
   a          5   17.0
   a          3    7.0
   b          2   45.0

### 1.2.1 DataFrame to TimeSeries conversion

#### 1.2.1.1 single value

In [5]:
ts = tspy.time_series.df(df, "timestamp", "value")
print(ts)

TimeStamp: 1     Value: 27.0
TimeStamp: 2     Value: 45.0
TimeStamp: 3     Value: 4.0
TimeStamp: 3     Value: 7.0
TimeStamp: 5     Value: 17.0


#### 1.2.1.2 multiple value

In [6]:
ts = tspy.time_series.df(df, "timestamp")
print(ts)

TimeStamp: 1     Value: {value=27.0, key=a}
TimeStamp: 2     Value: {value=45.0, key=b}
TimeStamp: 3     Value: {value=4.0, key=b}
TimeStamp: 3     Value: {value=7.0, key=a}
TimeStamp: 5     Value: {value=17.0, key=a}


### 1.2.2 TimeSeries to DataFrame conversion

In [7]:
# Similarly, you can convert back
ts.to_df().head()

timestamp  value key
0          1     27   a
1          2     45   b
2          3      4   b
3          3      7   a
4          5     17   a

### 1.2.3 DataFrame to MultiTimeSeries conversion

In [8]:
mts = tspy.multi_time_series.df_observations(df, "key", "timestamp", "value")
print(mts)

a time series
------------------------------
TimeStamp: 1     Value: 27.0
TimeStamp: 3     Value: 7.0
TimeStamp: 5     Value: 17.0
b time series
------------------------------
TimeStamp: 2     Value: 45.0
TimeStamp: 3     Value: 4.0


### 1.2.4 MultiTimeSeries to DataFrame conversion

In [9]:
# Similarly, you can convert back
mts.to_df_observations().head()

timestamp key  value
0          1   a     27
1          3   a      7
2          5   a     17
3          2   b     45
4          3   b      4

## 1.3 Data Retrieval

Since TimeSeries is actually a lazily evaluated object, it will not actually do any execution until explicitly prompted.

### 1.3.0 collect

TimeSeries collection is the simplist form of getting data in which the entire TimeSeries is brought in to memory

In [10]:
observations = ts.collect()
print(observations)

[(1,{value=27.0, key=a}),(2,{value=45.0, key=b}),(3,{value=4.0, key=b}),(3,{value=7.0, key=a}),(5,{value=17.0, key=a})]


### 1.3.1 get_values

TimeSeries also provides a _get_values_ method which will get values between a range. This can be useful if for instance, your data is very large as this will utilize narrow dependency.

In [11]:
observations = ts.get_values(2,4)
print(observations)

[(2,{value=45.0, key=b}),(3,{value=4.0, key=b}),(3,{value=7.0, key=a})]


### 1.3.2 exception handling

Because TimeSeries collection is lazy, most exceptions will be thrown a retrieval time. The following is how to handle those exceptions

In [12]:
from tspy.exceptions import TSError

# this is a faulty ts that should result in an error since len(int) has no meaning
ts = tspy.time_series.list([1,2,3]).map(lambda x: len(x))

try:
    ts.collect()
except TSError as e:
    print(e)

There was an issue collecting data, this may be caused by incorrect types given to chained operations


# 2. TimeSeries Segmentation

*Note: Each segment itself is actually a special form of materialized TimeSeries (collection of observations)*

## 2.0 Create a Basic TimeSeries

In [13]:
ts = tspy.time_series.list([1.0, 2.0, 3.0, 4.0, 5.0, 6.0])

## 2.1 Time Based Segmentation

In [14]:
sts = ts.segment_by_time(3, 3)
print(sts)

TimeStamp: 0     Value: original bounds: (0,2) actual bounds: (0,2) observations: [(0,1.0),(1,2.0),(2,3.0)]
TimeStamp: 3     Value: original bounds: (3,5) actual bounds: (3,5) observations: [(3,4.0),(4,5.0),(5,6.0)]


## 2.2 Record Based Segmentation

In [15]:
sts = ts.segment(2)
print(sts)

TimeStamp: 0     Value: original bounds: (0,1) actual bounds: (0,1) observations: [(0,1.0),(1,2.0)]
TimeStamp: 1     Value: original bounds: (1,2) actual bounds: (1,2) observations: [(1,2.0),(2,3.0)]
TimeStamp: 2     Value: original bounds: (2,3) actual bounds: (2,3) observations: [(2,3.0),(3,4.0)]
TimeStamp: 3     Value: original bounds: (3,4) actual bounds: (3,4) observations: [(3,4.0),(4,5.0)]
TimeStamp: 4     Value: original bounds: (4,5) actual bounds: (4,5) observations: [(4,5.0),(5,6.0)]


## 2.3 Anchor Based Segmentation

In [16]:
sts = ts.segment_by_anchor(lambda d: d % 2 == 0, 1, 1)
print(sts)

TimeStamp: 1     Value: original bounds: (0,2) actual bounds: (0,2) observations: [(0,1.0),(1,2.0),(2,3.0)]
TimeStamp: 3     Value: original bounds: (2,4) actual bounds: (2,4) observations: [(2,3.0),(3,4.0),(4,5.0)]
TimeStamp: 5     Value: original bounds: (4,6) actual bounds: (4,5) observations: [(4,5.0),(5,6.0)]


# 3. TimeSeries Transformation

*Note: In the context of this library a transform is an operation that given a TimeSeries, produces a new TimeSeries*

## 3.0 Create a Basic TimeSeries

In [17]:
ts = tspy.time_series.list([1.0, 2.0, 3.0, 4.0, 5.0, 6.0])

## 3.1 General TimeSeries Transforms

### 3.1.0 map

In [18]:
# Map each value in the TimeSeries to a new value + 1
add_one_ts = ts.map(lambda x: x + 1)
add_one_ts.print()

TimeStamp: 0     Value: 2.0
TimeStamp: 1     Value: 3.0
TimeStamp: 2     Value: 4.0
TimeStamp: 3     Value: 5.0
TimeStamp: 4     Value: 6.0
TimeStamp: 5     Value: 7.0


## 3.1.1 filter

In [19]:
even_ts = ts.filter(lambda x: x % 2 == 0)
even_ts.print()

TimeStamp: 1     Value: 2.0
TimeStamp: 3     Value: 4.0
TimeStamp: 5     Value: 6.0


### 3.1.2 lag

In [20]:
lag_ts = ts.lag(1)
lag_ts.print()

TimeStamp: 1     Value: 2.0
TimeStamp: 2     Value: 3.0
TimeStamp: 3     Value: 4.0
TimeStamp: 4     Value: 5.0
TimeStamp: 5     Value: 6.0


### 3.1.3 shift

In [21]:
shift_ts = ts.shift(1)
shift_ts.print()

TimeStamp: 0     Value: null
TimeStamp: 1     Value: 1.0
TimeStamp: 2     Value: 2.0
TimeStamp: 3     Value: 3.0
TimeStamp: 4     Value: 4.0
TimeStamp: 5     Value: 5.0


### 3.1.4 interpolation

In [22]:
from tspy.builders.functions import *
periodicity = 2
interp = interpolators.nearest(0.0)

interp_ts = ts.resample(periodicity, interp)
interp_ts.print()

TimeStamp: 0     Value: 1.0
TimeStamp: 2     Value: 3.0
TimeStamp: 4     Value: 5.0


### 3.1.5 fillna

In [23]:
shift_ts = ts.shift(2)

print("shifted ts to add nulls")
print(shift_ts)

print("\nfilled ts to make nulls 0s")
null_filled_ts = shift_ts.fillna(interpolators.fill(0.0))
print(null_filled_ts)

shifted ts to add nulls
TimeStamp: 0     Value: null
TimeStamp: 1     Value: null
TimeStamp: 2     Value: 1.0
TimeStamp: 3     Value: 2.0
TimeStamp: 4     Value: 3.0
TimeStamp: 5     Value: 4.0

filled ts to make nulls 0s
TimeStamp: 0     Value: 0.0
TimeStamp: 1     Value: 0.0
TimeStamp: 2     Value: 1.0
TimeStamp: 3     Value: 2.0
TimeStamp: 4     Value: 3.0
TimeStamp: 5     Value: 4.0


## 3.2 Complex (Math, Distance, Etc.) TimeSeries Transforms

*Note: This is not an exhaustive list*

### 3.2.0 difference

In [24]:
diff_ts = ts.transform(transformers.difference())
print(diff_ts)

TimeStamp: 1     Value: 1.0
TimeStamp: 2     Value: 1.0
TimeStamp: 3     Value: 1.0
TimeStamp: 4     Value: 1.0
TimeStamp: 5     Value: 1.0


### 3.2.1 z-normalization

In [25]:
z_norm_ts = ts.transform(transformers.z_score())
print(z_norm_ts)

TimeStamp: 0     Value: -1.4638501094227998
TimeStamp: 1     Value: -0.8783100656536799
TimeStamp: 2     Value: -0.29277002188455997
TimeStamp: 3     Value: 0.29277002188455997
TimeStamp: 4     Value: 0.8783100656536799
TimeStamp: 5     Value: 1.4638501094227998


### 3.2.2 add noise (AWGN)

In [26]:
noise_ts = ts.transform(transformers.awgn(mean=0.0,sd=.03))
print(noise_ts)

TimeStamp: 0     Value: 0.9984334710831994
TimeStamp: 1     Value: 1.9848526167613652
TimeStamp: 2     Value: 3.0236863702137047
TimeStamp: 3     Value: 4.0236130075691365
TimeStamp: 4     Value: 5.013298962013428
TimeStamp: 5     Value: 5.924005820832573


### 3.2.3 add noise (MWGN)

In [27]:
noise_ts = ts.transform(transformers.mwgn(.01))
print(noise_ts)

TimeStamp: 0     Value: 0.9962378841388397
TimeStamp: 1     Value: 1.9681980879378596
TimeStamp: 2     Value: 3.0289374962174405
TimeStamp: 3     Value: 3.990728648807705
TimeStamp: 4     Value: 4.935338359740761
TimeStamp: 5     Value: 6.03395072999318


### 3.2.4 remove consecutive duplicate values

In [28]:
with_dups_ts = tspy.time_series.list([1.0, 2.0, 2.0, 3.0, 4.0, 4.0, 5.0])
no_dups_ts = ts.transform(transformers.remove_consecutive_duplicate_values())
print(no_dups_ts)

TimeStamp: 0     Value: 1.0
TimeStamp: 1     Value: 2.0
TimeStamp: 2     Value: 3.0
TimeStamp: 3     Value: 4.0
TimeStamp: 4     Value: 5.0
TimeStamp: 5     Value: 6.0


### 3.2.5 combine duplicate timestamps

In [29]:
with_dups_ts = tspy.observations.of(
    tspy.observation(0, 1.0),
    tspy.observation(1, 2.0), 
    tspy.observation(1, 2.0), 
    tspy.observation(2, 3.0), 
    tspy.observation(3, 4.0), 
    tspy.observation(4, 4.0), 
    tspy.observation(3, 5.0)
).to_time_series()
no_dups_ts = with_dups_ts.transform(transformers.combine_duplicate_time_ticks(lambda x: sum(x)))
print(no_dups_ts)

TimeStamp: 0     Value: 1.0
TimeStamp: 1     Value: 4.0
TimeStamp: 2     Value: 3.0
TimeStamp: 3     Value: 9.0
TimeStamp: 4     Value: 4.0


# 4. TimeSeries Reduction

*Note: In the context of this library, a reducer is an operation that given a TimeSeries, produces a single value*

## 4.0 Create a Basic TimeSeries

In [30]:
ts = tspy.time_series.list([1.0, 2.0, 3.0, 4.0, 5.0, 6.0])

## 4.1 General TimeSeries Reductions

### 4.1.0 count

In [31]:
count = ts.count()
print(count)

6


### 4.1.1 describe

In [32]:
desc = ts.describe()
print(desc)

min inter-arrival-time: 1
max inter-arrival-time: 1
mean inter-arrival-time: 1.0
top: null
unique: 6
frequency: 1
first: TimeStamp: 0     Value: 1.0
last: TimeStamp: 5     Value: 6.0
count: 6
mean:3.5
std:1.707825127659933
min:1.0
max:6.0
25%:1.75
50%:3.5
75%:5.25



## 4.2 Complex TimeSeries Reductions

*Note: This is not an exhaustive list*

### 4.2.0 average

In [33]:
# Get the average over this TimeSeries
avg = ts.reduce(reducers.average())
print(avg)

3.5


### 4.2.1 auto-correlation

In [34]:
ac = ts.reduce(reducers.auto_correlation())
print(ac)

[0.5, 0.05714285714285725, -0.2714285714285713, -0.4285714285714287, -0.3571428571428572]


### 4.2.2 Dynamic Time Warping

In [35]:
ts2 = ts.transform(transformers.awgn(sd=.3))

dtw_distance = ts.reduce(ts2,
                         reducers.dtw(lambda obs1, obs2: abs(obs1.value - obs2.value)))
print(dtw_distance)

1.8557981638880405


### 4.2.3 Correlation

In [36]:
ts2 = ts.transform(transformers.awgn(sd=.3))

corr = ts.reduce(ts2, reducers.correlation())
print(corr)

0.9938941942380525


### 4.2.4 Augmented Dickey Fuller

In [37]:
adf = ts.reduce(reducers.adf())
print(adf)

pValue: -3.45
satisfies test: false


### 4.2.5 Granger Causality

In [38]:
ts2 = ts.transform(transformers.awgn(sd=.3))

granger = ts.reduce(ts2, reducers.granger(1))
print(granger)

-1.7123613937876463,-3.874412217575385,1.0


## 4.3 Segmentation and Reduction of TimeSeries

*Note: Since segmentation produces segments which are time series, you can perform a transform against a segmented time series using a reducer*

In [39]:
avg_ts = ts.segment(2).transform(reducers.average())
print(avg_ts)

TimeStamp: 0     Value: 1.5
TimeStamp: 1     Value: 2.5
TimeStamp: 2     Value: 3.5
TimeStamp: 3     Value: 4.5
TimeStamp: 4     Value: 5.5


# 5. Temporal Join/Align

## 5.0 Create Two Simple TimeSeries

In [40]:
# Create a collection of observations (materialized TimeSeries)
observations_left = tspy.observations.of(tspy.observation(1, 0.0), tspy.observation(3, 1.0), tspy.observation(8, 3.0), tspy.observation(9, 2.5))
observations_right = tspy.observations.of(tspy.observation(2, 2.0), tspy.observation(3, 1.5), tspy.observation(7, 4.0), tspy.observation(9, 5.5), tspy.observation(10, 4.5))

# Build TimeSeries from Observations
ts_left = observations_left.to_time_series()
ts_right = observations_right.to_time_series()

## 5.1 Temporal Join

*Note: We support all SQL join semantics, but for this example we are using Full join Semantics*

### 5.1.0 Default Join Parameters

*Note: This will fill missing values with null. The result of the join will be an array where the first value is from the left series and the second value is from the right series.*

In [41]:
# Perform full join
ts_full = ts_left.full_join(ts_right)
print(ts_full)

TimeStamp: 1     Value: [0.0, null]
TimeStamp: 2     Value: [null, 2.0]
TimeStamp: 3     Value: [1.0, 1.5]
TimeStamp: 7     Value: [null, 4.0]
TimeStamp: 8     Value: [3.0, null]
TimeStamp: 9     Value: [2.5, 5.5]
TimeStamp: 10     Value: [null, 4.5]


### 5.1.1 Join with interpolation

*Note: This will fill missing values based on the type of interpolation. The result of the join will be an array where the first value is from the left series and the second value is from the right series.*

In [42]:
# Perform full join with interpolation
interp = interpolators.linear()

ts_full = ts_left.full_join(ts_right, left_interp_func=interp, right_interp_func=interp)
print(ts_full)

TimeStamp: 1     Value: [0.0, null]
TimeStamp: 2     Value: [0.5, 2.0]
TimeStamp: 3     Value: [1.0, 1.5]
TimeStamp: 7     Value: [2.6, 4.0]
TimeStamp: 8     Value: [3.0, 4.75]
TimeStamp: 9     Value: [2.5, 5.5]
TimeStamp: 10     Value: [null, 4.5]


### 5.1.2 Join with interpolation and custom mapping

*Note: This will fill missing values based on the type of interpolation. The result on the join will be decided based on the given lambda function.*

In [43]:
# Perform full join with interpolation and mapping
interp = interpolators.linear(0.0, 1, 1)
mapping = lambda l,r: l + r

ts_full = ts_left.full_join(ts_right, mapping, interp, interp)
print(ts_full)

TimeStamp: 1     Value: 0.0
TimeStamp: 2     Value: 2.5
TimeStamp: 3     Value: 2.5
TimeStamp: 7     Value: 6.6
TimeStamp: 8     Value: 7.75
TimeStamp: 9     Value: 8.0
TimeStamp: 10     Value: 4.5


## 5.2 Temporal Align

*Note: We support all SQL join semantics, but for this example we are using left join Semantics*

### 5.2.0 Default Align

*Note: Missing values will be filled with nulls.*

In [44]:
# Perform left align
ts_left_aligned, ts_right_aligned = ts_left.left_align(ts_right)

print("left ts result")
print(ts_left_aligned)
print("right ts result")
print(ts_right_aligned)

left ts result
TimeStamp: 1     Value: 0.0
TimeStamp: 3     Value: 1.0
TimeStamp: 8     Value: 3.0
TimeStamp: 9     Value: 2.5
right ts result
TimeStamp: 1     Value: null
TimeStamp: 3     Value: 1.5
TimeStamp: 8     Value: null
TimeStamp: 9     Value: 5.5


### 5.2.1 Align with interpolation

*Note: Missing values will be filled based on the type of interpolation given.*

In [45]:
# Perform left align with interpolation
ts_left_aligned, ts_right_aligned = ts_left.left_align(ts_right, interpolators.nearest(0.0))

print("left ts result")
print(ts_left_aligned)
print("right ts result")
print(ts_right_aligned)

left ts result
TimeStamp: 1     Value: 0.0
TimeStamp: 3     Value: 1.0
TimeStamp: 8     Value: 3.0
TimeStamp: 9     Value: 2.5
right ts result
TimeStamp: 1     Value: 0.0
TimeStamp: 3     Value: 1.5
TimeStamp: 8     Value: 4.0
TimeStamp: 9     Value: 5.5


# 6. Forecasting

## 6.1 Forecasting Model

TimeSeries library currently supports the following forecasting models:

- BATS
- Holt-Winters

*Note: More will be added in later updates*

### 6.1.0 Model Creation

#### 6.1.0.0 BATS

In [46]:
training_sample_size = 10
bats_model = tspy.forecasters.bats(10)
print(bats_model)

Forecasting Model
  Algorithm: BATSAlgorithm


#### 6.1.0.1 Holt-Winters

In [47]:
samples_per_season = 5
initial_training_seasons = 10
hws_model = tspy.forecasters.hws(samples_per_season=samples_per_season, initial_training_seasons=initial_training_seasons)
print(hws_model)

Forecasting Model
  Algorithm: HWSAdditive=5 (aLevel=0, bSlope=0, gSeas=0) uninitialized


### 6.1.1 Model Updating

In [48]:
import random

model = tspy.forecasters.hws(samples_per_season=samples_per_season, initial_training_seasons=initial_training_seasons)

for i in range(100):
    timestamp = i
    value = random.randint(1,10) * 1.0
    model.update_model(timestamp, value)

print(model)

Forecasting Model
  Algorithm: HWSAdditive=5 (aLevel=0.001, bSlope=0.001, gSeas=0.001) level=6.087789839896166, slope=0.018901997884893912, seasonal(amp,per,avg)=(1.411203455586738,5, 0,-0.0037471500727535465)


### 6.1.2 Model Forecasting

Once your model is initialized, you can then forecast. 

*Note: It is always good practice to first check if your model is initialized*

In [49]:
if model.is_initialized():
    print(model.forecast_at(120))

6.334135728495107


## 6.2 Forecasting Model in TimeSeries

### 6.2.0 Create a simple TimeSeries Object

In [50]:
ts = tspy.time_series.list([float(i) for i in range(10)])

print(ts)

TimeStamp: 0     Value: 0.0
TimeStamp: 1     Value: 1.0
TimeStamp: 2     Value: 2.0
TimeStamp: 3     Value: 3.0
TimeStamp: 4     Value: 4.0
TimeStamp: 5     Value: 5.0
TimeStamp: 6     Value: 6.0
TimeStamp: 7     Value: 7.0
TimeStamp: 8     Value: 8.0
TimeStamp: 9     Value: 9.0


In [51]:
num_predictions = 5
model = tspy.forecasters.auto(8)
confidence = .99

predictions = ts.forecast(num_predictions, model, confidence=confidence)

print(predictions.to_time_series())

TimeStamp: 10     Value: {value=10.0, lower_bound=10.0, upper_bound=10.0, error=0.0}
TimeStamp: 11     Value: {value=10.997862810553725, lower_bound=9.934621260488143, upper_bound=12.061104360619307, error=0.41277640121597475}
TimeStamp: 12     Value: {value=11.996821082897318, lower_bound=10.704895525154571, upper_bound=13.288746640640065, error=0.5015571318964149}
TimeStamp: 13     Value: {value=12.995779355240911, lower_bound=11.50957896664928, upper_bound=14.481979743832543, error=0.5769793776877866}
TimeStamp: 14     Value: {value=13.994737627584504, lower_bound=12.33653268707341, upper_bound=15.652942568095598, error=0.6437557559526337}


In [52]:
print(predictions.to_time_series().to_df())

   timestamp      value  lower_bound  upper_bound     error
0         10  10.000000    10.000000    10.000000  0.000000
1         11  10.997863     9.934621    12.061104  0.412776
2         12  11.996821    10.704896    13.288747  0.501557
3         13  12.995779    11.509579    14.481980  0.576979
4         14  13.994738    12.336533    15.652943  0.643756


In [53]:
print(model)

Forecasting Model
  Algorithm: HWSAdditive:HWSAdditive=1 (aLevel=0.001, bSlope=0.001, gSeas=0.001) level=9.99004488020975, slope=0.9999900896408388, seasonal(amp,per,avg)=(0.0,1, 4,-0.009900448802097483)
HWSMultiplicative:HWSMultiplicative=1 (aLevel=0.001, bSlope=0.001, gSeas=0.001) level=9.990109117210386, slope=0.9999901537464366, seasonal(amp,per,avg)=(0.0,1, 4,0.9970972695898375)
RegularARIMAAlgorithm:RegularARIMAAlgorithm [armaAlgorithm=RegularARMAAlgorithm [dataHistory=null, errorHistory=null, dataMean=NaN, minTrainingData=80, pMin=0, pMax=5, qMin=0, qMax=5, forceModel=false], differencer=null, minTrainingData=82, diffOrder=-1]
RegularSegmentedLinear:RegularSegmentedLinear=1,history=-1,width=1 [(y=1.0*x + 0.0)  ]
RegularSegmentedAveraging:RegularSegmentedAveraging=1,history=-1,width=1 [(avg=4.5)  ]
RegularLinear:RegularLinear [linearFitter=LeastSquaresFitter [yIntercept=0.0, slope=1.0], minTrainingSamples=2]
RegularLinear:RegularLinear [linearFitter=LeastSquaresFitter [yIntercept